## Job aggregators

Job aggregators are essentially search engines for job ads. Job aggregators gather job postings from job boards and other internet sites and consolidate them into a single searchable interface ; Web scrap naukri.com and indeed portal and give an single searchable interface

### Requirements

In [ ]:
!pip install requests
!pip install bs4

In [ ]:
!pip install selenium

In [3]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

### Webscraping using selenium

In [ ]:
role = input("Enter role: ")
loc = input("Enter loc: ")

driver1 = webdriver.Chrome()
driver2 = webdriver.Chrome()

# URL1 = "https://www.naukri.com/big-data-architect-jobs-in-bangalore-bengaluru?k=big%20data%20architect&l=bangalore%2Fbengaluru"
URL1 = f'https://www.naukri.com/big-data-architect-jobs-in-bangalore-bengaluru?k={role}&l={loc}'

# URL2 = https://in.indeed.com/jobs?q=therapist&l=Coimbatore
URL2 = f'https://in.indeed.com/jobs?q={role}&l={loc}'

driver1.get(URL1)
soup1 = BeautifulSoup(driver1.page_source, "html.parser")
# print(soup1.prettify())
driver1.close()

driver2.get(URL2)
soup2 = BeautifulSoup(driver2.page_source, "html.parser")
# print(soup2.prettify())
driver2.close()

### Getting the appropriate tags

In [45]:
naukri = pd.DataFrame(columns=['Title','Company','Ratings','Reviews','URL'])
results1 = soup1.find(class_='list')
job_elems1 = results1.find_all('article',class_='jobTuple bgWhite br4 mb-8')

In [46]:
indeed = pd.DataFrame(columns=['Title','Company','Ratings','Reviews','URL'])
results2 = soup2.find(class_='mosaic mosaic-provider-jobcards mosaic-provider-hydrated')
job_elems2 = results2.find_all('a', attrs={"data-hide-spinner":"true"})

# <a  class="tapItem fs-unmask result job_69866087ffb93cd0 resultWithShelf sponTapItem desktop" href="/company/shine/jobs/Spa-Therapist-69866087ffb93cd0?fccid=55832d3e98b164b6&amp;vjs=3" target="_blank"><div class="slider_container"><div class="slider_list"><div class="slider_item"><div class="job_seen_beacon"><table class="jobCard_mainContent" cellpadding="0" cellspacing="0" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple"><span title="Spa Therapist">Spa Therapist</span></h2></div><div class="heading6 company_location tapItem-gutter"><pre><span class="companyName">shine</span><div class="companyLocation">Coimbatore, Tamil Nadu</div></pre></div><div class="heading6 tapItem-gutter metadataContainer"><div class="metadata salary-snippet-container"><div aria-label="₹9,030 a month" class="salary-snippet"><span>₹9,030 a month</span></div></div></div><div class="heading6 error-text tapItem-gutter"></div></td></tr></tbody></table><table class="jobCardShelfContainer" role="presentation"><tbody><tr class="jobCardShelf"><td class="shelfItem indeedApply"><span class="iaIcon"></span><span class="ialbl iaTextBlack">Apply securely with Indeed Resume</span></td><td class="shelfItem urgentlyHiring"><div class="urgentlyHiring"><span class="urgentlyHiringIcon"></span>Urgently hiring</div></td></tr><tr class="underShelfFooter"><td><div class="heading6 tapItem-gutter result-footer"><div class="job-snippet"><ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;"> 

### Building the Dataframes

In [47]:
# Building the dataframe for Indeed

for job_elem in job_elems2:

	# URL
	url_prefix = "https://in.indeed.com"
	URL = job_elem.attrs['href']
	URL = url_prefix + URL

	# Job title
	Title = job_elem.find('div',class_='heading4 color-text-primary singleLineTitle tapItem-gutter')
	Title = Title.text

	if Title[0:3]=='new':
		Title=Title[3:]

	# Company
	Company = job_elem.find('div',class_='heading6 company_location tapItem-gutter')
	Company = Company.find('span',class_='companyName').text

	#Ratings
	rating = job_elem.find('div',class_='heading6 company_location tapItem-gutter')
	rating_span = rating.find('span',class_='ratingNumber')
	if rating_span is None:
			Ratings = "NA"
	else:
			Ratings = rating_span.text

	# Salary offered for the job
	Sal_span = job_elem.find('div',class_='salary-snippet')
	if Sal_span is None:
			Salary = "NA"
	else:
			Salary = Sal_span.text

	# Location for the job post
	Loc = job_elem.find('div',class_='heading6 company_location tapItem-gutter')
	Loc_exp = Loc.find('div',class_='companyLocation')
	if Loc_exp is None:
			Location = "NA"
	else:
			Location = Loc_exp.text

	# Number of days since job posted
	Hist = job_elem.find('div','heading6 tapItem-gutter result-footer')
	Post_Hist = Hist.find('span',class_='date')
	if Post_Hist is None:
			Post_History = "NA"
	else:
			Post_History = Post_Hist.text

	indeed=indeed.append({'URL':URL,'Title':Title,'Company':Company,'Salary':Salary,'Location':Location,'Post_History':Post_History,'Ratings':Ratings},ignore_index = True)
	

In [50]:
# Building the dataframe for Naukri

for job_elem in job_elems1:

        # URL to apply for the job     
        URL = job_elem.find('a',class_='title fw500 ellipsis').get('href')

        # Post Title
        Title = job_elem.find('a',class_='title fw500 ellipsis')

        # Company Name
        Company = job_elem.find('a',class_='subTitle ellipsis fleft')

        # Ratings
        rating_span = job_elem.find('span',class_='starRating fleft dot')
        if rating_span is None:
            Ratings = "NA"
        else:
            Ratings = rating_span.text

        # Reviews Counts
        Review_span = job_elem.find('a',class_='reviewsCount ml-5 fleft blue-text')
        if Review_span is None:
            Reviews = "NA"
        else:
            Reviews = Review_span.text

        # Years of experience Required
        Exp = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi experience')
        Exp_span = Exp.find('span',class_='ellipsis fleft fs12 lh16')
        if Exp_span is None:
            Experience = "NA"
        else:
            Experience = Exp_span.text

        # Salary offered for the job
        Sal = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi salary')
        Sal_span = Sal.find('span',class_='ellipsis fleft fs12 lh16')
        if Sal_span is None:
            Salary = "NA"
        else:
            Salary = Sal_span.text

        # Location for the job post
        Loc = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi location')
        Loc_exp = Loc.find('span',class_='ellipsis fleft fs12 lh16')
        if Loc_exp is None:
            Location = "NA"
        else:
            Location = Loc_exp.text

        # Number of days since job posted
        Hist = job_elem.find("div",["type br2 fleft grey","type br2 fleft green"])
        Post_Hist = Hist.find('span',class_='fleft fw500')
        if Post_Hist is None:
            Post_History = "NA"
        else:
            Post_History = Post_Hist.text

        naukri = naukri.append({'URL':URL,'Title':Title.text,'Company':Company.text,'Ratings':Ratings,'Reviews':Reviews,'Experience':Experience,'Salary':Salary,'Location':Location,'Job_Post_History':Post_History},ignore_index = True)

### Printing the Dataframes

In [ ]:
print(naukri.shape, indeed.shape)

In [55]:
naukri

,Title,Company,Ratings,Reviews,URL,Experience,Job_Post_History,Location,Salary
0,ID&A - Data Analyst - Informatica MDM,Shell,4.1,(715 Reviews),https://www.naukri.com/job-listings-id-a-data-...,5-9 Yrs,2 Days Ago,Bangalore/Bengaluru,Not disclosed
1,Sr Business Data Analyst,Enquero,4.0,(26 Reviews),https://www.naukri.com/job-listings-sr-busines...,4-7 Yrs,6 Days Ago,Bangalore/Bengaluru,Not disclosed
2,Business Data Analyst - Database Design/Mining,AugmatrixGo,NA,NA,https://www.naukri.com/job-listings-business-d...,2-5 Yrs,2 Days Ago,Bangalore/Bengaluru,Not disclosed
3,Business Data Analyst,NetApp,4.2,(122 Reviews),https://www.naukri.com/job-listings-business-d...,5-10 Yrs,5 Days Ago,Bangalore/Bengaluru,Not disclosed
4,Senior Data Analyst - KPO,Huquo Consulting Pvt. Ltd,NA,NA,https://www.naukri.com/job-listings-senior-dat...,7-12 Yrs,2 Days Ago,"Gurgaon/Gurugram, Bangalore/Bengaluru",Not disclosed
5,Senior Data Analyst - SQL/Tableau/Redshift,Pronto Consulting Services,NA,NA,https://www.naukri.com/job-listings-senior-dat...,NA,4 Days Ago,NA,Not disclosed
6,Data Analyst with SQL,Wipro,3.8,(13978 Reviews),https://www.naukri.com/job-listings-data-analy...,3-7 Yrs,2 Days Ago,"Noida, Gurgaon/Gurugram, Bangalore/Bengaluru",Not disclosed
7,Capital Market- Data Analyst,Genpact,4.0,(11489 Reviews),https://www.naukri.com/job-listings-capital-ma...,4-8 Yrs,2 Days Ago,"Hyderabad/Secunderabad, Pune, Bangalore/Bengal...",Not disclosed
8,Sr Business Data Analyst,ExecBoardinAsia,NA,NA,https://www.naukri.com/job-listings-sr-busines...,5-10 Yrs,9 Days Ago,Bangalore/Bengaluru,Not disclosed
9,Customer Acquisition Engineer (Data Analyst),OT-SOLUTIONS TECH PRIVATE LIMITED,NA,NA,https://www.naukri.com/job-listings-customer-a...,0-2 Yrs,2 Days Ago,"Indore, Bangalore/Bengaluru, Delhi / NCR","3,00,000 - 5,00,000 PA."


In [56]:
indeed

,Title,Company,Ratings,Reviews,URL,Location,Post_History,Salary
0,Junior Data Analyst,Chesa Dental Care Services Limited,NA,NaN,https://in.indeed.com/company/Chesa-Dental-Car...,"Bengaluru, Karnataka",3 days ago,"₹22,000 a month"
1,Hiring Data Analyst and Strategic Reporting - ...,Enhance IT,NA,NaN,https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"Bengaluru, Karnataka•Remote",12 days ago,"₹7,00,000 - ₹12,00,000 a year"
2,Data Analyst,Eduauraa Technologies Pvt Ltd,NA,NaN,https://in.indeed.com/company/Eduauraa-Technol...,"Bengaluru, Karnataka",4 days ago,"₹25,000 - ₹35,000 a month"
3,Excel Operator - Intern Data Analyst,AIMLEAP,NA,NaN,https://in.indeed.com/company/AIMLEAP/jobs/Exc...,"Bengaluru, Karnataka•Temporarily Remote",7 days ago,"₹8,000 - ₹10,000 a month"
4,Sustainability Data Analyst | 4 to 6 years | M...,Capgemini,3.8,NaN,https://in.indeed.com/rc/clk?jk=8eb85bf92c08d5...,"Bengaluru, Karnataka",6 days ago,NA
5,Data Analyst,Flipkart,4.1,NaN,https://in.indeed.com/rc/clk?jk=761a8e4be1548a...,"Bengaluru, Karnataka",10 days ago,NA
6,Data Analyst,CultYvate,NA,NaN,https://in.indeed.com/company/CultYvate/jobs/D...,"Bengaluru, Karnataka",2 days ago,"₹25,000 a month"
7,Data Analyst,Cityinfo Services Private Limited,NA,NaN,https://in.indeed.com/company/Cityinfo-Service...,"Bengaluru, Karnataka",11 days ago,"₹13,031 - ₹18,000 a month"
8,Data Analyst,Myntra.com,4.2,NaN,https://in.indeed.com/rc/clk?jk=e624c2687065be...,"Bengaluru, Karnataka",5 days ago,NA
9,Data Analyst,Applied Materials Inc.,3.9,NaN,https://in.indeed.com/rc/clk?jk=482575da974e08...,"Bengaluru, Karnataka",4 days ago,NA


The end.